In [2]:
import requests
import pandas as pd
import datetime
import time
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.copy_on_write = True 

In [3]:
#Функция парсинга поисковой формы
def parser (name_text, tz_text, data_from, date_to, okdp):
    #Создаю папку с временем парсинга
    today = datetime.datetime.now()
    d1 = today.strftime("%d-%m-%Y %H-%M-%S")
    path_in_file = 'data/'+d1
    if not os.path.exists(path_in_file):
        os.makedirs(path_in_file)
    #Сохранение 10 порций по 500
    for i in range(10):
        params = {
            #Параметры запроса
            'searchString' : f'{name_text}',                          #Поиск по названию (слово или фраза)
            'morphology' : 'on',                          #Морфология поиска (on или off)
            'search-filter' : 'Дате размещения',          #поиск по: 'Дате размещения', 'Дате обновления'
            'pageNumber' : '1',                           #Страница выдачи (в нашем случае не учитывается)
            'sortDirection' : 'false',                    #Сортировка выдачи (по убыванию/по возрастанию)
            'recordsPerPage' : '_50',                     #Количество на странице выдачи (в нашем случае не учитывается)
            'showLotsInfoHidden' : 'false',               #Что-то связанное с лотами (пока не разобрался)
            'sortBy' : 'UPDATE_DATE',                     #Сортировка выдачи (По дате, размещения, по дате обновления, по цене, по релевантности)
            'fz44' : 'on',                                #Искать по ФЗ-44 (on или off)
            'fz223' : 'on',                               #Искать по ФЗ-223 (on или off)
            'af' : 'on',                                  #Этап подача заявок (on или off)
            'ca' : 'on',                                  #Этап работа комиссии (on или off)
            'pc' : 'on',                                  #Этап закупка завершена(on или off)
            'currencyIdGeneral' : '-1',                   #Тип цены
            'publishDateFrom' : f'{data_from}',             #С даты публикации
            'publishDateTo' : f'{date_to}',               #По дату публикации
            'okpd2IdsWithNested': 'on',                   #Включить вложенные ОКПД2 (on или off)
            'okpd2Ids' : '8873937,8873938',
            'okpd2IdsCodes': f'{okdp}',                     #ОКПД2
            'gws' : 'Выберите тип закупки',
            'OrderPlacementSmallBusinessSubject' : 'on',  #Закупки для малого бизнеса (on или off)
            'OrderPlacementRnpData' : 'on',
            'OrderPlacementExecutionRequirement' : 'on',
            'orderPlacement94_0' : '0',                   #Что-то для ФЗ-94
            'orderPlacement94_1' : '0',                   #Что-то для ФЗ-94
            'orderPlacement94_2' : '0',                   #Что-то для ФЗ-94
            'searchTextInAttachedFile' : f'{tz_text}',              #Поиск по названию (слово или фраза)
            #Доступна выгрузка не более 5000 закупок порциями по 500
            'from' : f'{str(500*i + 1)}',                 #Начало порции
            'to' : f'{str(500 + 500*i)}',                 #Конец порции
            #Поля включаемые в CSV (true|false)
            'placementCsv' : 'false',                      #ФЗ
            'registryNumberCsv' : 'true',                 #Реестровый номер закупки
            'stepOrderPlacementCsv' : 'false',             #Этап закупки
            'methodOrderPurchaseCsv' : 'false',            #Способ определения поставщика
            'nameOrderCsv' : 'true',                      #Наименование закупки
            'purchaseNumbersCsv' : 'false',                #Идентификационный код закупки
            'numberLotCsv' : 'false',                      #Номер лота
            'nameLotCsv' : 'false',                        #Наименование лота
            'maxContractPriceCsv' : 'false',               #Начальная (максимальная) цена контракта
            'currencyCodeCsv' : 'false',                   #Валюта
            'maxPriceContractCurrencyCsv' : 'false',      #Доп Валюта
            'currencyCodeContractCurrencyCsv' : 'false',  #Доп Валюта
            'scopeOkdpCsv' : 'false',                     #Старые ОКПД
            'scopeOkpdCsv' : 'false',                     #Старые ОКПД
            'scopeOkpd2Csv' : 'true',                     #ОКПД2
            'scopeKtruCsv' : 'false',                      #ОКПД2
            'ea615ItemCsv' : 'false',                      #Предмет электронного аукциона (только для ПП РФ 615)
            'customerNameCsv' : 'false',                   #Наименование Заказчика
            'organizationOrderPlacementCsv' : 'false',     #Организация, осуществляющая размещение 
            'publishDateCsv' : 'true',                    #Дата размещения
            'lastDateChangeCsv' : 'false',                 #Дата обновления
            'startDateRequestCsv' : 'false',               #Дата начала подачи заявок
            'endDateRequestCsv' : 'false',                 #Дата окончания подачи заявок
            'ea615DateCsv' : 'false',                      #Дата проведения электронного аукциона (только для ПП РФ 615)
            'featureOrderPlacementCsv' : 'false'           #Особенности размещения заказа\ Преимущества (44-ФЗ)
            }
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 YaBrowser/23.11.0.0 Safari/537.36'
        }
        url = 'https://zakupki.gov.ru/epz/order/orderCsvSettings/download.html?'
        r = requests.get(url, headers = headers, params = params, timeout = 30)
        time.sleep(2)
        f = open(f'{path_in_file}/order{str(i + 1)}.csv', 'w')
        f.write(r.text)
        f.close()
        time.sleep(2)
    #Обединение порций в один датафрейм
    d = dict()
    frame = []
    for i in range(10):
        d['df_%s' % i] = pd.read_csv(f'{path_in_file}/order{str(i + 1)}.csv', sep =';', encoding='cp1251')
        frame.insert(i, d['df_%s' % i])
    result = pd.concat(frame) 
    return result

In [4]:
#Задаю параметры для парсинга
name_text = ''
tz_text = ''
data_from = '01.01.2024'
date_to = '31.01.2024'
okdp = '62,63'

In [5]:
#Создание датафрейма со спарсенными данными
df = parser(name_text, tz_text, data_from, date_to, okdp)
df.reset_index(drop= True , inplace= True )

In [6]:
#Итоговый датафрейм
df

,Реестровый номер закупки,Наименование закупки,Классификация по ОКПД2,Дата размещения
0,№0352300014224000006,Оказание услуги по адаптации и сопровождению э...,62.02.30.000: Услуги по технической поддержке ...,25.01.2024
1,№0372200177824000002,Оказание услуг по адаптации и сопровождению эк...,62.02.20.190: Услуги консультативные в области...,25.01.2024
2,№0848300046124000004,Оказание услуг по размещению информации об осн...,"63.99.1: Услуги информационные прочие, не вклю...",16.01.2024
3,№0318200044424000001,Сертификат на техническую поддержку программно...,62.02.3: Услуги по технической поддержке инфор...,26.01.2024
4,№0350400000424000001,Предоставление лицензии на антивирус для нужд ...,62.01.29.000: Оригиналы программного обеспечен...,24.01.2024
...,...,...,...,...
3651,№32413155177,"Сопровождение программных продуктов ПО ""Парус-...",62.03.12.130: Услуги по сопровождению компьюте...,04.01.2024
3652,№32413155162,"Сопровождение программных продуктов фирмы ""1С""",62.03.12.130: Услуги по сопровождению компьюте...,04.01.2024
3653,№32413154964,Информационные услуги систем Консультант+,62.09.20.190: Услуги по технической поддержке ...,03.01.2024
3654,№32413154922,Поставка нефтепродуктов и оказание сервисных ...,63.99.10.190: Услуги информационные автоматизи...,03.01.2024


In [149]:
df.to_csv('df.csv')